In [4]:
def parse_mapping_data(lines):
    # Parses the mapping data from the lines.
    mappings = []
    i = 2
    while i < len(lines):
        current_map = []
        i += 1
        while i < len(lines) and lines[i] != "":
            dst_start, src_start, range_length = map(int, lines[i].split())
            current_map.append((dst_start, src_start, range_length))
            i += 1
        mappings.append(current_map)
        i += 1
    return mappings

def find_location(seed, mappings):
    # Finds the location number for a given seed.
    current_number = seed
    for mapping in mappings:
        for dst_start, src_start, range_length in mapping:
            if src_start <= current_number < src_start + range_length:
                current_number = dst_start + (current_number - src_start)
                break
    return current_number

# Reading the data from the file
with open("raw.txt") as raw:
    lines = raw.read().strip().split("\n")

# Parsing seeds and mappings
seeds = list(map(int, lines[0].split(" ")[1:]))
mappings = parse_mapping_data(lines)

# Finding locations for each seed and determining the minimum location
locations = [find_location(seed, mappings) for seed in seeds]
print(min(locations))


196167384


In [6]:
def parse_seed_pairs(lines):
    """Parses seed pairs from the first line of the file."""
    raw_seeds = list(map(int, lines[0].split(" ")[1:]))
    return [(raw_seeds[i], raw_seeds[i + 1]) for i in range(0, len(raw_seeds), 2)]

def parse_mappings(lines):
    """Parses mapping data from the file lines."""
    mappings = []
    i = 2
    while i < len(lines):
        _, _, _ = lines[i].split(" ")[0].split("-")  # Extracting category names (unused)
        mapping = []
        i += 1
        while i < len(lines) and lines[i] != "":
            dst_start, src_start, range_len = map(int, lines[i].split())
            mapping.append((dst_start, src_start, range_len))
            i += 1
        mapping.sort(key=lambda x: x[1])
        mappings.append(mapping)
        i += 1
    return mappings

def remap_interval(lo, hi, mapping):
    """Remaps an interval (lo, hi) using a given mapping."""
    remapped = []
    for dst, src, length in mapping:
        end = src + length - 1
        shift = dst - src
        if not (end < lo or src > hi):
            remapped.append((max(src, lo), min(end, hi), shift))

    for i, (l, r, shift) in enumerate(remapped):
        yield (l + shift, r + shift)
        if i < len(remapped) - 1 and remapped[i+1][0] > r + 1:
            yield (r + 1, remapped[i+1][0] - 1)

    if len(remapped) == 0:
        yield (lo, hi)
    elif remapped[0][0] != lo:
        yield (lo, remapped[0][0] - 1)
    elif remapped[-1][1] != hi:
        yield (remapped[-1][1] + 1, hi)

# Reading data from the file
with open("raw.txt") as raw:
    lines = raw.read().strip().split("\n")

# Parsing seeds and mappings
seeds = parse_seed_pairs(lines)
mappings = parse_mappings(lines)

# Finding minimum location
min_location = float('inf')
for start, range_len in seeds:
    current_intervals = [(start, start + range_len - 1)]
    for mapping in mappings:
        new_intervals = [new_interval for lo, hi in current_intervals for new_interval in remap_interval(lo, hi, mapping)]
        current_intervals = new_intervals
    min_location = min(min_location, min(lo for lo, hi in current_intervals))

print(min_location)


125742456
